In [1]:
# pip install datasets

In [2]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/us-patent-phrase-to-phrase-matching/sample_submission.csv
/kaggle/input/us-patent-phrase-to-phrase-matching/train.csv
/kaggle/input/us-patent-phrase-to-phrase-matching/test.csv


In [3]:
data = pd.read_csv('/kaggle/input/us-patent-phrase-to-phrase-matching/train.csv')
data.head()

,id,anchor,target,context,score
0,37d61fd2272659b1,abatement,abatement of pollution,A47,0.50
1,7b9652b17b68b7a4,abatement,act of abating,A47,0.75
2,36d72442aefd8232,abatement,active catalyst,A47,0.25
3,5296b0c19e1ce60e,abatement,eliminating process,A47,0.50
4,54c1e3b9184cb5b6,abatement,forest region,A47,0.00


In [4]:
data.shape

(36473, 5)

In [5]:
data.describe(include='object')

,id,anchor,target,context
count,36473,36473,36473,36473
unique,36473,733,29340,106
top,37d61fd2272659b1,component composite coating,composition,H01
freq,1,152,24,2186


In [6]:
data['input'] = 'TEXT1:'+ data.context + 'TEXT2:' + data.target + 'ANC:' + data.anchor
data.input.head(10)

0    TEXT1:A47TEXT2:abatement of pollutionANC:abate...
1           TEXT1:A47TEXT2:act of abatingANC:abatement
2          TEXT1:A47TEXT2:active catalystANC:abatement
3      TEXT1:A47TEXT2:eliminating processANC:abatement
4            TEXT1:A47TEXT2:forest regionANC:abatement
5         TEXT1:A47TEXT2:greenhouse gasesANC:abatement
6           TEXT1:A47TEXT2:increased rateANC:abatement
7        TEXT1:A47TEXT2:measurement levelANC:abatement
8        TEXT1:A47TEXT2:minimising soundsANC:abatement
9    TEXT1:A47TEXT2:mixing core materialsANC:abatement
Name: input, dtype: object

In [7]:
from datasets import Dataset, DatasetDict
df = Dataset.from_pandas(data)
df

Dataset({
    features: ['id', 'anchor', 'target', 'context', 'score', 'input'],
    num_rows: 36473
})

In [8]:
model = 'microsoft/deberta-v3-small'

In [9]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification
tokenz = AutoTokenizer.from_pretrained(model, padding='max_len')

Downloading:   0%|          | 0.00/52.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/578 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/2.35M [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
/opt/conda/lib/python3.7/site-packages/transformers/convert_slow_tokenizer.py:435: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  "The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option"
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [10]:
tokenz.tokenize('Hi! I am really excited about the $uperman: Legacy film for the DCU')

['▁Hi',
 '!',
 '▁I',
 '▁am',
 '▁really',
 '▁excited',
 '▁about',
 '▁the',
 '▁$',
 'up',
 'erman',
 ':',
 '▁Legacy',
 '▁film',
 '▁for',
 '▁the',
 '▁DCU']

In [11]:
tokenz.tokenize('Okay, but will Ras Al Ghul or Slade Wilson conjure up in Brave and the Bold????')

['▁Okay',
 ',',
 '▁but',
 '▁will',
 '▁Ras',
 '▁Al',
 '▁G',
 'hul',
 '▁or',
 '▁Slade',
 '▁Wilson',
 '▁conjure',
 '▁up',
 '▁in',
 '▁Brave',
 '▁and',
 '▁the',
 '▁Bold',
 '?',
 '?',
 '?',
 '?']

In [12]:
def tokenize(x): 
    return tokenz(x['input'])

In [13]:
tok_df = df.map(tokenize, batched=True)

  0%|          | 0/37 [00:00<?, ?ba/s]

In [14]:
row = tok_df[0]
row['input'], row['input_ids']

('TEXT1:A47TEXT2:abatement of pollutionANC:abatement',
 [1,
  54453,
  435,
  294,
  558,
  5753,
  104917,
  445,
  294,
  16191,
  297,
  17019,
  265,
  6435,
  64097,
  294,
  16191,
  297,
  17019,
  2])

In [15]:
tok_df = tok_df.rename_columns({'score':'labels'})

In [16]:
test_df = pd.read_csv('/kaggle/input/us-patent-phrase-to-phrase-matching/test.csv')
test_df.describe()

,id,anchor,target,context
count,36,36,36,36
unique,36,34,36,29
top,4112d61851461f60,el display,inorganic photoconductor drum,G02
freq,1,2,1,3


In [17]:
dds = tok_df.train_test_split(0.25, seed=42)
dds

DatasetDict({
    train: Dataset({
        features: ['id', 'anchor', 'target', 'context', 'labels', 'input', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 27354
    })
    test: Dataset({
        features: ['id', 'anchor', 'target', 'context', 'labels', 'input', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 9119
    })
})

In [18]:
test_df['input'] = 'TEXT1:'+ test_df.context + 'TEXT2:' + test_df.target + 'ANC:' + test_df.anchor
test_df = Dataset.from_pandas(test_df).map(tokenize, batched=True)

  0%|          | 0/1 [00:00<?, ?ba/s]

In [19]:
def corr(x,y):
    return np.corrcoef(x,y)[0][1]

In [20]:
def corr_d(test_df):
    return {'pearson': corr(*test_df)}

In [21]:
# pip install transformers

In [22]:
from transformers import Trainer, TrainingArguments
bs = 128
epochs=4
lr = 7e-5

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [23]:
args = TrainingArguments('outputs', learning_rate=lr, warmup_ratio=0.1,
                         lr_scheduler_type='cosine', fp16 = True,
                         evaluation_strategy='epoch', per_device_train_batch_size=bs,
                         per_device_eval_batch_size=bs*2, num_train_epochs=epochs,
                         weight_decay=0.01, report_to='none'
                        )



In [24]:
model_x = AutoModelForSequenceClassification.from_pretrained(model, num_labels=1)
trainer = Trainer(model_x, args, train_dataset=dds['train'], eval_dataset=dds['test'],
                 tokenizer=tokenz, compute_metrics=corr_d)

Downloading:   0%|          | 0.00/273M [00:00<?, ?B/s]

Some weights of the model checkpoint at microsoft/deberta-v3-small were not used when initializing DebertaV2ForSequenceClassification: ['lm_predictions.lm_head.LayerNorm.weight', 'lm_predictions.lm_head.bias', 'lm_predictions.lm_head.LayerNorm.bias', 'lm_predictions.lm_head.dense.weight', 'mask_predictions.dense.weight', 'mask_predictions.classifier.bias', 'mask_predictions.LayerNorm.weight', 'mask_predictions.classifier.weight', 'mask_predictions.LayerNorm.bias', 'lm_predictions.lm_head.dense.bias', 'mask_predictions.dense.bias']
- This IS expected if you are initializing DebertaV2ForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DebertaV2ForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from 

In [25]:
trainer.train()

The following columns in the training set don't have a corresponding argument in `DebertaV2ForSequenceClassification.forward` and have been ignored: anchor, target, context, id, input. If anchor, target, context, id, input are not expected by `DebertaV2ForSequenceClassification.forward`,  you can safely ignore this message.
/opt/conda/lib/python3.7/site-packages/transformers/optimization.py:310: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,
***** Running training *****
  Num examples = 27354
  Num Epochs = 4
  Instantaneous batch size per device = 128
  Total train batch size (w. parallel, distributed & accumulation) = 256
  Gradient Accumulation steps = 1
  Total optimization steps = 428
/opt/conda/lib/python3.7/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along

Epoch,Training Loss,Validation Loss,Pearson
1,No log,0.030035,0.770418
2,No log,0.025430,0.802152
3,No log,0.025958,0.809363
4,No log,0.025484,0.813551


The following columns in the evaluation set don't have a corresponding argument in `DebertaV2ForSequenceClassification.forward` and have been ignored: anchor, target, context, id, input. If anchor, target, context, id, input are not expected by `DebertaV2ForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 9119
  Batch size = 512
The following columns in the evaluation set don't have a corresponding argument in `DebertaV2ForSequenceClassification.forward` and have been ignored: anchor, target, context, id, input. If anchor, target, context, id, input are not expected by `DebertaV2ForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 9119
  Batch size = 512
The following columns in the evaluation set don't have a corresponding argument in `DebertaV2ForSequenceClassification.forward` and have been ignored: anchor, target, context, id, input. If anchor, ta

TrainOutput(global_step=428, training_loss=0.028975544688857605, metrics={'train_runtime': 172.613, 'train_samples_per_second': 633.88, 'train_steps_per_second': 2.48, 'total_flos': 676715015824740.0, 'train_loss': 0.028975544688857605, 'epoch': 4.0})

In [26]:
preds = trainer.predict(test_df).predictions.astype('float')
preds = np.clip(preds, 0,1)
preds

The following columns in the test set don't have a corresponding argument in `DebertaV2ForSequenceClassification.forward` and have been ignored: anchor, target, context, id, input. If anchor, target, context, id, input are not expected by `DebertaV2ForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Prediction *****
  Num examples = 36
  Batch size = 512


array([[0.56152344],
       [0.77441406],
       [0.49169922],
       [0.38647461],
       [0.15258789],
       [0.57275391],
       [0.58007812],
       [0.00980377],
       [0.32006836],
       [1.        ],
       [0.34057617],
       [0.28369141],
       [0.77001953],
       [0.73876953],
       [0.78369141],
       [0.56738281],
       [0.27539062],
       [0.        ],
       [0.65771484],
       [0.31591797],
       [0.33544922],
       [0.23950195],
       [0.02313232],
       [0.2590332 ],
       [0.61083984],
       [0.        ],
       [0.        ],
       [0.        ],
       [0.        ],
       [0.69482422],
       [0.29760742],
       [0.04949951],
       [0.73095703],
       [0.52587891],
       [0.4934082 ],
       [0.20471191]])

In [27]:
import datasets

submission = datasets.Dataset.from_dict({
    'id': test_df['id'],
    'score': preds
})

submission.to_csv('submission.csv', index=False)

Creating CSV from Arrow format:   0%|          | 0/1 [00:00<?, ?ba/s]

1039